# Tratamento de Dados IPCA - IBGE

Este notebook realiza o tratamento dos dados do IPCA (Índice de Preços ao Consumidor Amplo) do IBGE, incluindo limpeza, ordenação e padronização dos dados.

## 1. Importação de Bibliotecas

In [15]:
import pandas as pd
import gzip
from io import StringIO
import os

## 2. Configuração de Caminhos e Arquivos

In [16]:
# Estrutura de diretórios (Clean Architecture)
diretorio_dados_brutos = '../../data/raw/'
diretorio_dados_tratados = '../../data/processed/'
diretorio_relatorios = '../../reports/'

# Criando os diretórios se não existirem
for diretorio in [diretorio_dados_brutos, diretorio_dados_tratados, diretorio_relatorios]:
    os.makedirs(diretorio, exist_ok=True)
    print(f"Diretório verificado: {diretorio}")

# Configuração dos arquivos
arquivo_original = os.path.join(diretorio_dados_brutos, 'br_ibge_ipca_mes_brasil.csv.gz')
arquivo_tratado = os.path.join(diretorio_dados_tratados, 'ipca_processado.csv')

# Verificar se o arquivo existe
if os.path.exists(arquivo_original):
    print(f"Arquivo encontrado: {arquivo_original}")
    tamanho_mb = os.path.getsize(arquivo_original) / (1024 * 1024)
    print(f"Tamanho: {tamanho_mb:.2f} MB")
else:
    print(f"Arquivo não encontrado: {arquivo_original}")
    print("Por favor, coloque o arquivo 'br_ibge_ipca_mes_brasil.csv.gz' na pasta 'data/raw/'")
    
    # Verificando se existe na raiz (para compatibilidade)
    if os.path.exists('../../br_ibge_ipca_mes_brasil.csv.gz'):
        print("Encontrado arquivo na raiz do projeto - será usado temporariamente")
        arquivo_original = '../../br_ibge_ipca_mes_brasil.csv.gz'

Diretório verificado: ../../data/raw/
Diretório verificado: ../../data/processed/
Diretório verificado: ../../reports/
Arquivo encontrado: ../../data/raw/br_ibge_ipca_mes_brasil.csv.gz
Tamanho: 0.01 MB


## 3. Função de Carregamento e Tratamento dos Dados

In [17]:
def carregar_e_tratar(nome_arquivo_comprimido):
    """
    Carrega e trata os dados do IPCA a partir de um arquivo .csv.gz
    
    Parâmetros:
    nome_arquivo_comprimido (str): nome do arquivo comprimido
    
    Retorna:
    pandas.DataFrame: dados tratados do IPCA
    """
    # Lendo o arquivo comprimido
    with gzip.open(nome_arquivo_comprimido, 'rt') as arquivo:
        conteudo = arquivo.read()
    
    # Tratando valores nulos
    conteudo = conteudo.replace('..', '0')
    
    # Convertendo para DataFrame
    df = pd.read_csv(StringIO(conteudo))
    
    # Limpeza e formatação dos dados
    df = df.dropna(how='all')  # Remove linhas completamente vazias
    
    # Converte colunas numéricas
    colunas_numericas = df.select_dtypes(include=['object']).columns
    for col in colunas_numericas:
        if col not in ['Código', 'Mês', 'Local']:
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                pass
    
    # Ordena por ano e mês se as colunas existirem
    if 'Ano' in df.columns and 'Mês' in df.columns:
        df = df.sort_values(['Ano', 'Mês'])
    
    print(f"Dados carregados: {len(df)} registros")
    print(f"Colunas: {list(df.columns)}")
    
    return df

## 4. Execução do Processamento

In [18]:
# Carregando e tratando os dados
try:
    df_ipca = carregar_e_tratar(arquivo_original)
    print("\n Dados carregados com sucesso!")
    
    # Exibindo informações sobre os dados
    print(f"\nShape dos dados: {df_ipca.shape}")
    print(f"Primeiras 5 linhas:")
    print(df_ipca.head())
    
except Exception as e:
    print(f" Erro ao carregar os dados: {e}")

Dados carregados: 540 registros
Colunas: ['ano', 'mes', 'indice', 'variacao_mensal', 'variacao_trimestral', 'variacao_semestral', 'variacao_anual', 'variacao_doze_meses']

 Dados carregados com sucesso!

Shape dos dados: (540, 8)
Primeiras 5 linhas:
    ano  mes        indice  variacao_mensal  variacao_trimestral  \
0  2000    1  1.598410e+03             0.62                 2.19   
1  2016    1  4.550230e+03             1.27                 3.27   
2  1985    1  5.656761e-07            11.76                38.32   
3  1997    1  1.379330e+03             1.18                 1.98   
4  2014    1  3.836370e+03             0.55                 2.02   

   variacao_semestral  variacao_anual  variacao_doze_meses  
0                4.30            0.62                 8.85  
1                4.91            1.27                10.71  
2               86.66           11.76               221.27  
3                2.89            1.18                 9.39  
4                3.21            0.5

## 5. Análise Exploratória dos Dados

In [19]:
# Análise exploratória dos dados
if 'df_ipca' in locals():
    print(" ANÁLISE EXPLORATÓRIA DOS DADOS IPCA")
    print("=" * 50)
    
    # Informações gerais
    print(f"Número de registros: {len(df_ipca)}")
    print(f"Número de colunas: {len(df_ipca.columns)}")
    print(f"Colunas: {list(df_ipca.columns)}")
    
    # Verificando valores nulos
    print(f"\n Valores nulos por coluna:")
    nulos = df_ipca.isnull().sum()
    for col, count in nulos.items():
        if count > 0:
            print(f"  {col}: {count} ({count/len(df_ipca)*100:.1f}%)")
    
    # Informações sobre tipos de dados
    print(f"\n Tipos de dados:")
    print(df_ipca.dtypes)
    
    # Estatísticas descritivas para colunas numéricas
    colunas_numericas = df_ipca.select_dtypes(include=['int64', 'float64']).columns
    if len(colunas_numericas) > 0:
        print(f"\n  Estatísticas descritivas:")
        print(df_ipca[colunas_numericas].describe())

 ANÁLISE EXPLORATÓRIA DOS DADOS IPCA
Número de registros: 540
Número de colunas: 8
Colunas: ['ano', 'mes', 'indice', 'variacao_mensal', 'variacao_trimestral', 'variacao_semestral', 'variacao_anual', 'variacao_doze_meses']

 Valores nulos por coluna:
  variacao_mensal: 1 (0.2%)
  variacao_trimestral: 3 (0.6%)
  variacao_semestral: 6 (1.1%)
  variacao_anual: 1 (0.2%)
  variacao_doze_meses: 12 (2.2%)

 Tipos de dados:
ano                      int64
mes                      int64
indice                 float64
variacao_mensal        float64
variacao_trimestral    float64
variacao_semestral     float64
variacao_anual         float64
variacao_doze_meses    float64
dtype: object

  Estatísticas descritivas:
               ano         mes        indice  variacao_mensal  \
count   540.000000  540.000000  5.400000e+02       539.000000   
mean   2001.916667    6.500000  2.292682e+03         5.700724   
std      13.002158    3.455253  2.114032e+03        10.679030   
min    1979.000000    1.000000

## 6. Exportação dos Dados Tratados

In [20]:
# Salvando os dados tratados
if 'df_ipca' in locals():
    try:
        df_ipca.to_csv(arquivo_tratado, index=False)
        print(f"Dados salvos com sucesso em: {arquivo_tratado}")
        print(f"Tamanho do arquivo: {os.path.getsize(arquivo_tratado)} bytes")
        
        # Verificando se o arquivo foi criado
        if os.path.exists(arquivo_tratado):
            print(f"Arquivo {arquivo_tratado} criado com sucesso!")
            
            # Lendo as primeiras linhas do arquivo salvo para verificação
            df_verificacao = pd.read_csv(arquivo_tratado, nrows=3)
            print(f"Verificação - Primeiras 3 linhas do arquivo salvo:")
            print(df_verificacao)
            
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")
else:
    print("Dados não foram carregados. Execute as células anteriores primeiro.")

Dados salvos com sucesso em: ../../data/processed/ipca_processado.csv
Tamanho do arquivo: 23808 bytes
Arquivo ../../data/processed/ipca_processado.csv criado com sucesso!
Verificação - Primeiras 3 linhas do arquivo salvo:
    ano  mes        indice  variacao_mensal  variacao_trimestral  \
0  2000    1  1.598410e+03             0.62                 2.19   
1  2016    1  4.550230e+03             1.27                 3.27   
2  1985    1  5.656761e-07            11.76                38.32   

   variacao_semestral  variacao_anual  variacao_doze_meses  
0                4.30            0.62                 8.85  
1                4.91            1.27                10.71  
2               86.66           11.76               221.27  
